In [1]:
import os
import time
import numpy as np
import pandas as pd
from numpy import newaxis
from sklearn.model_selection import train_test_split
import glob
import matplotlib.pyplot as plt
import PIL

In [2]:
import tensorflow as tf

print(f"tensorflow version: {tf.__version__}")
print(f"tensorflow version: {tf.keras.__version__}")

tensorflow version: 2.5.0
tensorflow version: 2.5.0


In [57]:
# Import normal Dataset
def import_dataset(filename = "./Dataset/csv/Attacked/Original_X_test.csv"):
    with open(filename, newline='') as csvfile:
        rows = pd.read_csv(csvfile,header=None)
        X_train, X_test = train_test_split(rows, test_size=0.2)
        X_train = np.array(X_train)
        X_test = np.array(X_test)
        X_train_ex = X_train[:, :, newaxis]
        X_test_ex = X_test[:, :, newaxis]
    return X_train, X_test, X_train_ex, X_test_ex

In [58]:
X_train, X_test, X_train_ex, X_test_ex = import_dataset()

In [27]:
import ctypes
bin(ctypes.c_uint64.from_buffer(ctypes.c_double(1C6C000000000000)).value)

'0b100000001110100101000000000000000000000000000000000000000000000'

In [38]:
print(X_train[2])
for i in X_train[2]:
    print("{0:b}".format(int(i)))

[5.39e-03 5.01e+02 0.00e+00 1.00e+02 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00]
0
111110101
0
1100100
0
0
0
0
0
0


In [63]:
from keras.models import Sequential

input_img = tf.keras.layers.Input(shape = (X_train.shape[1],))
encoder = tf.keras.layers.Dense(10, activation='relu')(input_img)
encoder = tf.keras.layers.Dense(8, activation='relu')(encoder)
encoder = tf.keras.layers.Dense(2, activation='relu', name="latent_space")(encoder)

decoder = tf.keras.layers.Dense(8, activation='relu')(encoder)
decoder = tf.keras.layers.Dense(10, activation='relu')(decoder)
decoder = tf.keras.layers.Dense(X_train.shape[1], activation='relu')(decoder)

autoencoder = tf.keras.models.Model(input_img, decoder)
autoencoder.compile(loss='binary_crossentropy', optimizer= 'adam')

autoencoder.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_18 (Dense)             (None, 8)                 88        
_________________________________________________________________
latent_space (Dense)         (None, 2)                 18        
_________________________________________________________________
dense_19 (Dense)             (None, 8)                 24        
_________________________________________________________________
dense_20 (Dense)             (None, 10)                90        
_________________________________________________________________
dense_21 (Dense)             (None, 10)                110 

In [47]:
autoencoder.fit(X_train ,X_train ,epochs=10, batch_size=256,validation_data=(X_test, X_test))

Epoch 1/10
140/140 [==============================] - 1s 5ms/step - loss: 447.8049 - val_loss: 450.0454
Epoch 2/10
140/140 [==============================] - 1s 4ms/step - loss: 447.8049 - val_loss: 450.0454
Epoch 3/10
140/140 [==============================] - 1s 4ms/step - loss: 447.8048 - val_loss: 450.0454
Epoch 4/10
140/140 [==============================] - 1s 4ms/step - loss: 447.8050 - val_loss: 450.0454
Epoch 5/10
140/140 [==============================] - 1s 4ms/step - loss: 447.8049 - val_loss: 450.0454
Epoch 6/10
140/140 [==============================] - 1s 4ms/step - loss: 447.8050 - val_loss: 450.0454
Epoch 7/10
140/140 [==============================] - 1s 4ms/step - loss: 447.8048 - val_loss: 450.0454
Epoch 8/10
140/140 [==============================] - 1s 4ms/step - loss: 447.8049 - val_loss: 450.0454
Epoch 9/10
140/140 [==============================] - 1s 4ms/step - loss: 447.8049 - val_loss: 450.0454
Epoch 10/10
140/140 [==============================] - 1s 4ms/st

In [48]:
decode_msg = autoencoder.predict(X_test)

In [51]:
decode_msg[0]

array([ 0.      ,  0.      ,  0.      , 12.861246, 34.474308, 31.388147,
        0.      , 44.982544, 17.23376 , 31.459887], dtype=float32)

In [52]:
X_test[0]

array([1.91e-06, 3.30e+02, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00])

In [83]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Embedding, GlobalAveragePooling1D, Flatten
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv1D,MaxPool1D

tf.compat.v1.disable_eager_execution()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
# model = Sequential()
# model.add(Conv1D(filters=100, kernel_size=5, padding='same',activation='relu',strides=1, input_shape=(X_train_ex.shape[1],1))) 
# model.add(Dropout(0.2))
# model.add(Conv1D(filters=200, kernel_size=5, padding='same', activation='relu',strides=1))
# model.add(Dropout(0.2))
# model.add(Conv1D(filters=400, kernel_size=10, padding='same', activation='relu',strides=1))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(Dropout(0.2))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])
# model.summary()

input = tf.keras.layers.Input(shape = (X_train_ex.shape[1],1))
encoder = tf.keras.layers.Conv1D(filters=100, kernel_size=5, padding='same',activation='relu',strides=1)(input)
# encoder = tf.keras.layers.MaxPool1D(pool_size=2)(encoder)
encoder = tf.keras.layers.Conv1D(filters=50, kernel_size=5, padding='same',activation='relu',strides=1)(encoder)
# encoder = tf.keras.layers.MaxPool1D(pool_size=2)(encoder)

decoder = tf.keras.layers.Conv1D(filters=50, kernel_size=5, padding='same',activation='relu',strides=1)(encoder)
# decoder = tf.keras.layers.MaxPool1D(pool_size=2)(decoder)
decoder = tf.keras.layers.Conv1D(filters=100, kernel_size=5, padding='same',activation='relu',strides=1)(decoder)
#decoder = tf.keras.layers.MaxPool1D(pool_size=2)(decoder)
decoder = tf.keras.layers.Dense(1, activation='relu')(decoder)
          

conv_autoencoder = tf.keras.models.Model(input, decoder)
conv_autoencoder.compile(optimizer = 'Adam', loss = 'binary_crossentropy')
conv_autoencoder.summary()


Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 10, 1)]           0         
_________________________________________________________________
conv1d_76 (Conv1D)           (None, 10, 100)           600       
_________________________________________________________________
conv1d_77 (Conv1D)           (None, 10, 50)            25050     
_________________________________________________________________
conv1d_78 (Conv1D)           (None, 10, 50)            12550     
_________________________________________________________________
conv1d_79 (Conv1D)           (None, 10, 100)           25100     
_________________________________________________________________
dense_31 (Dense)             (None, 10, 1)             101       
Total params: 63,401
Trainable params: 63,401
Non-trainable params: 0
______________________________________________________

In [84]:
conv_autoencoder.fit(X_train_ex, X_train_ex, epochs=10, batch_size=128, validation_data=(X_test_ex, X_test_ex))

Train on 35784 samples, validate on 8946 samples
Epoch 1/10


UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv1d_76/conv1d}}]]
	 [[loss_5/mul/_291]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv1d_76/conv1d}}]]
0 successful operations.
0 derived errors ignored.